In [8]:
import numpy as np
# from cv2 import putText,imencode,FONT_HERSHEY_SIMPLEX,LINE_4,resize, Canny, addWEight
import cv2
import time
import threading
import os
# append system.path with the folder containing basisklassen_cam,.py
import sys
sys.path.append('./../Software')
from basisklassen_cam import Camera

In [9]:
import ipywidgets as widgets
from IPython.display import display

## Einfache Verwendung von widget.Image

In diesem Fall wird demonstriert wie ein Bild als NumPy-Array einem Objekt vom Type `Image` übergeben wird.

In [10]:
img = cv2.imread('squirrel.jfif')
_, img_jpg = cv2.imencode('.jpeg', img)
img_jpg_bytes = img_jpg.tobytes()
image = widgets.Image(value=img_jpg_bytes)
display(image)
#

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In einer Schleife kann dieses Bild ständig aktualisiert werden. Der Beispielcode wird endlos ausgeführt. Zur Beendigung muss die Ausführung der Zelle beendet werden. Geben Sie danach auch die Kamera frei!

In [15]:
cam = Camera()
cam.release()

[ WARN:0] global ../opencv/modules/videoio/src/cap_v4l.cpp (890) open VIDEOIO(V4L2:/dev/video0): can't open camera by index


In [16]:

# Verwendung der Kamera uas basisklassen_cam.py
cam = Camera(width=64*3, height=48*3) # Erstellen der Kamera
print(cam.get_size())
#cam.release()
image = widgets.Image()            # Erstellen eines Widgets Image
display(image)                     # Anzeigen des Widgets

# Schleife aktualisiert Image
while True:
    frame = cam.get_frame()
    _, jpg = cv2.imencode('.jpeg', frame)
    image.value = jpg.tobytes()
    if  ord() == ord('q'):
        break

cam.release()

(0, 0)


[ WARN:0] global ../opencv/modules/videoio/src/cap_v4l.cpp (890) open VIDEOIO(V4L2:/dev/video0): can't open camera by index


Image(value=b'')

Exception: Could not open video device

In [ ]:
cam.release()

In [ ]:
image.close()

## Verwendung eines Threads für das Aktualisieren des Bildes
Dies erlaubt die Verwendung z.B. eines Umschalters (Toggle) zur Beendigung der Darstellung. Ohne die Verwendung der Thread wird der Event des Umschaltens nicht erkannt.

In [ ]:
image = widgets.Image()
stopButton = widgets.ToggleButton(value=True,description='Stop',disabled=False,icon='stop')
display(image,stopButton)
cam = Camera(height = 38*5, width = 64*5)

def threadFunc():
    while stopButton.value:
        try: #Es soll sichergestellt werden das der Thread beendet wird!
            frame = cam.get_frame()
            _, jpg = cv2.imencode('.jpeg', frame)
            image.value = jpg.tobytes()
        except:
            print('Thread cancelled by exception!')
            break
    image.close()
    stopButton.close()
    cam.release()

thread = threading.Thread(target = threadFunc)
thread.start()

## Ein Beispielwidget

In [ ]:
# Widget erlaubt die Anzeige eines Bildes/Live-Streams in einer Output-Zelle
# Dem Widget werden zwei Funktionen übergeben. Diese Funktionen müssen Numpy-Arrays zurückgeben, welche Bilder darstellen.
# Im Widget wird ein Thread gestartet, welcher in einer Schleife diese Funktionen kontinuiertlich aufruft und so die Bilder im Widget aktualisiert.
# Dieses Widget ist bewusst nicht direkt an eine Kamera gebunden, sondern nur an die zwei besagten Funktionen, welche Bilder zurückgeben.
 
class Display:
    def __init__(self, frameFuncA = None, frameFuncB = None, textFunc = None, size=(240,320),scalingFactor=1):
        self.imageSize=(int(size[1]*scalingFactor),int(size[0]*scalingFactor))
        self.getFrameA = frameFuncA
        self.getFrameB = frameFuncB
        self.textFunc = textFunc
        #Elements
        self.emptyFrameAsJpegBytes = cv2.imencode('.jpeg',np.ones((self.imageSize[1],self.imageSize[0],3))*200)[1].tobytes()
        self.imageA = widgets.Image(value = self.emptyFrameAsJpegBytes)
        self.imageB = widgets.Image(value = self.emptyFrameAsJpegBytes)
        #self.output = widgets.Output()
        self.text = widgets.Text(
                value='',
                description = 'Info:',
                disabled=False,
                width='auto'
        )
        self.stopButton = widgets.ToggleButton(
            value=True,
            description='Pause',
            disabled=False,
            icon='pause' 
        )
        def observe_stopButton(change):
            print(change)
            if self.stopButton.value == False:
                self.stopButton.description = 'Play'
                self.stopButton.icon = 'play'
            else: 
                self.stopButton.description = 'Pause'
                self.stopButton.icon = 'pause'
        self.stopButton.observe(observe_stopButton)
        self.closeButton = widgets.ToggleButton(
            value=True,
            description='Close Display',
            disabled=False,
            icon='times-circle'
        )
        def observe_closeButton(change):
            self.close()
        self.closeButton.observe(observe_closeButton)

        #Layout
        buttons = [
                    self.stopButton,
                    self.closeButton,
                ]
        if self.textFunc:
            buttons.append(self.text)
        self.layout = widgets.VBox([
            widgets.HBox([
                    widgets.HTML(value="<b>Display</b> {}".format(str(self.imageSize))),
                ]), 
            widgets.HBox([
                widgets.VBox([
                    widgets.HTML(value="<b>Frame A</b>"),
                    self.imageA,
                ]),
                widgets.VBox([
                    widgets.HTML(value="<b>Frame B</b>"),
                    self.imageB,
                ]),                     
            ]),
            widgets.HBox(buttons), 
            ])
        display(self.layout)
        self.thread = threading.Thread(target=self.run)
        self.start()

    def run(self):
        while self.flag:
            try:
                if self.stopButton.value==True:
                    if self.getFrameA is not None: 
                        frame = self.getFrameA()
                        frame = cv2.resize(frame, self.imageSize, 0)#interpolation = cv2.INTER_AREA)
                        _, jpg = cv2.imencode('.jpeg', frame)
                        self.imageA.value=jpg.tobytes()
                    if self.getFrameB is not None: 
                        frame = self.getFrameB()
                        frame = cv2.resize(frame, self.imageSize, 0)# interpolation = cv2.INTER_AREA)
                        _, jpg = cv2.imencode('.jpeg', frame)
                        self.imageB.value=jpg.tobytes()
                    if self.textFunc:
                        self.text.value = self.textFunc()
                else:
                    pass
            except Exception as error:
                print('Error:',error)
                self.layout.close()
                break
        print('Thread finished')

    def close(self):
        self.flag = False
        self.layout.close()

    def start(self):
        self.flag = True
        self.thread.start()

Definition einiger Funktionen und Verwendung mit der Klasse Display. Es werden nicht die selben Frames im Display dargestellt.

In [ ]:
cam = Camera(height = 380, width = 640)

# 1.Funktion
f = cam.get_frame # Übergabe der Referenz der Funktion cam.get_frame

# 2.Funktion
def g():
    
    frame = cam.get_frame()
    h,w,c = frame.shape     
    mask = cv2.Canny(frame,100,200)
    mask_inv = cv2.bitwise_not(mask)
    mask_bgr = cv2.cvtColor(mask,cv2.COLOR_GRAY2BGR)
    frame2 = cv2.bitwise_and(frame,frame,mask = mask_inv)
    frame3 = cv2.addWeighted(frame2,1,mask_bgr,1,0)
    ih = int(.1*h)
    frame3[:ih] = (frame3[:ih]/3)
    frame3[-ih:] = (frame3[-ih:]/3)
    return frame3

# 3. Funktion für text
def h():
    return 'nothing to report'

# Dem Konstruktor der Klasse Display werden zwei Funktionen übergeben.
d = Display(f,g,h)

In [ ]:
print('end of cell')
cam.release()

Verwendung der Klasse Display mit einem Objekt, dass verschieden Bilder per Methoden zurückliefert.
Es werden verschiedene Varianten des gleichen Frames dargestellt.

In [ ]:
class MyObj:
    def __init__(self):
        self.__cam = Camera(height = 380, width = 640)
        self.__last_frame = None
        self.__transformer_function = lambda x: x
        self.__counter = 0
        
    def get_frame(self):
        self.__last_frame = self.__cam.get_frame()
        self.__counter += 1
        self.__last_frame_transformed = self.__transformer_function(self.__last_frame)
        return self.__last_frame
        
    def get_last_transformed_frame(self):
        return self.__last_frame_transformed
    
    def get_last_frame(self):
        return self.__last_frame
    
    def get_counter(self):
        return self.__counter
    
    def set_transformer(self,transformer_function):
        self.__transformer_function = transformer_function
        
def my_transformer_to_gray(x):
    return cv2.cvtColor(x,cv2.COLOR_BGR2GRAY)

def my_transformer(frame):
    _,w,_ = frame.shape
    iw=int(w/2)
    frame_canny = cv2.Canny(frame,100,200)
    frame_canny_bgr = cv2.cvtColor(frame_canny,cv2.COLOR_GRAY2BGR)
    frame_canny_bgr[:,-iw:]=frame[:,-iw:]
    return frame_canny_bgr

In [ ]:
myobj = MyObj()
myobj.set_transformer(my_transformer)
Display(myobj.get_frame, myobj.get_last_transformed_frame, lambda : str(myobj.get_counter()))

In [ ]:
myobj.set_transformer(lambda x:x)

In [ ]:
myobj.set_transformer(my_transformer)

In [ ]:
cam.release()